In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import os

from collections import Counter

# Final Panel Dataset

In [2]:
# Panel Function

def panel(data, name):
    
    problem_orgs = [' Inc.', ' LLC', ' Inc ', ' LLP ', ' LLP', ' Ltd.', ' P.C.', ' L.P.', ' LP ', ' Inc', ' LP', ' S.A.', ' PC ', ' PLLC', ' plc ']
    
    leadership_id = []
    first_name = []
    last_name = []
    title = []
    org_name = []
    start_date = []
    end_date = []
    full_jobs = []
    other_current = []
    
    #Panelization on past careers
    
    df = data.drop_duplicates('Leadership People Id') # Dropping any duplicates of 'Leadership People Id'
    df = df[df['Careers'].notnull()].reset_index() # Selection of a sub dataset with non null careers for panelization

    for i in tqdm(range(len(df['Careers']))): # Generation of a new line for each past career
        
        leadership_id.append(df.loc[i, 'Leadership People Id'])
        first_name.append(df.loc[i, 'First Name'])
        last_name.append(df.loc[i, 'Last Name'])
        title.append(df.loc[i, 'Title'])
        org_name.append(df.loc[i, 'Organization Name (Parent)'])
        full_jobs.append(str(df.loc[i, 'Title']) + ', ' + str(df.loc[i, 'Organization Name (Parent)']))
        start_date.append(df.loc[i, 'Start Date'])
        end_date.append(None)
        other_current.append(0)
        
        l = df.loc[i, 'Careers'].split(';') #Split of 'Çareers' into the single past careers
        for j in range(len(l)):
            
            l2 = l[j].split(',') # Split of the single career into Title (l2[0]) and Organization Name + Dates (l2[-1])
            
            if l2[-1][:2] == '  ':
                pass
            
            else:
                
                full_jobs.append(l[j])
                if l2[-1][:5] in problem_orgs or l2[-1][:4] in problem_orgs or l2[-1][:3] in problem_orgs: 
                    new = ['', '']
                    new[0] = l2[0]
                    new[1] = ''.join(l2[-2:])
                    l2 = new

                #Generation of the columns
                leadership_id.append(df.loc[i, 'Leadership People Id'])
                first_name.append(df.loc[i, 'First Name'])
                last_name.append(df.loc[i, 'Last Name'])
                title.append(l2[0])

                if '(' not in l2[-1]: #Then neither a start date nor an end date
                    org_name.append(l2[-1])
                    start_date.append(None)
                    end_date.append(None)
                else:
                    if l2[-1][-6] == '(': #Only start date
                        org_name.append(l2[-1][:-6])
                        start_date.append(l2[-1][-5:-1])
                        end_date.append(None)
                    else: #Both start and end date
                        org_name.append(l2[-1][:-11])
                        start_date.append(l2[-1][-10:-6])
                        end_date.append(l2[-1][-5:-1])
                other_current.append(0)
            
    #Panelization on other current roles

    df = data.drop_duplicates('Leadership People Id') # Dropping any duplicates of 'Leadership People Id'
    df = df[df['Other Current Roles'].notnull()].reset_index()
    
    for k in tqdm(range(len(df['Other Current Roles']))): # Generation of a new line for each other current role
        
        if str(df.loc[k, 'Careers']) == 'nan':
            leadership_id.append(df.loc[k, 'Leadership People Id'])
            first_name.append(df.loc[k, 'First Name'])
            last_name.append(df.loc[k, 'Last Name'])
            title.append(df.loc[k, 'Title'])
            org_name.append(df.loc[k, 'Organization Name (Parent)'])
            full_jobs.append(str(df.loc[k, 'Title']) + ', ' + str(df.loc[k, 'Organization Name (Parent)']))
            start_date.append(df.loc[k, 'Start Date'])
            end_date.append(None)
            other_current.append(0) 
        
        l = df.loc[k,'Other Current Roles'].split(';') #Split of 'Other Current Roles' into the single current roles
        for h in range(len(l)):
            
            l2 = l[h].split(',') # Split of the single career into Title (l2[0]) and Organization Name + Dates (l2[-1])

            if l2[-1][:2] == '  ':
                pass
            
            else:
                
                full_jobs.append(l[h])
                if l2[-1][:5] in problem_orgs or l2[-1][:4] in problem_orgs or l2[-1][:3] in problem_orgs: 
                    new = ['', '']
                    new[0] = l2[0]
                    new[1] = ''.join(l2[-2:])
                    l2 = new

                #Generation of the columns
                leadership_id.append(df.loc[k,'Leadership People Id'])
                first_name.append(df.loc[k,'First Name'])
                last_name.append(df.loc[k,'Last Name'])
                title.append(l2[0])
                start_date.append(df.loc[k,'Start Date'])
                end_date.append(None)
                org_name.append(l2[-1])
                other_current.append(1)
            
    
    #Generation of the dataframe with the generated columns
    df3 = pd.DataFrame()
    df3['Leadership People Id'] = leadership_id
    df3['First Name'] = first_name
    df3['Last Name'] = last_name
    df3['Title'] = title
    df3['Organization Name (Parent)'] = org_name
    df3['Full Job'] = full_jobs
    df3['Start Date'] = start_date
    df3['End Date'] = end_date
    df3['Other Current'] = other_current
    
    df = data[(data['Careers'].notnull()) | (data['Other Current Roles'].notnull())]
    df = df.rename({'Start Date':'Current Start Date'}, axis = 1)
    
    #The new dataset is merged with the original dataset to keep all the other time invariant individuals' characteristics
    cols_2 = ['Leadership People Id'] + list(set(df.columns) - set(df3.columns))
    df5 = pd.merge(df.loc[:, cols_2], df3, on = 'Leadership People Id')
    final_cols = list(data.columns) + ['Full Job', 'Current Start Date', 'Other Current', 'End Date']
    df5 = df5[final_cols]
    
    # Finally, the new dataset is concatenated with the observations from the original dataset whose 'Careers' and 'Other Current Roles' column is null, which
    # have only their current career and no past career neither other current roles (thus no need of panelization)
    df4 = data.drop_duplicates('Leadership People Id')
    final_df = pd.concat([df5, df4[(df4['Careers'].isnull()) & (df4['Other Current Roles'].isnull())]])
    final_df.to_csv(name)

In [4]:
companies_person = pd.read_csv('Companies_Person.csv')
companies_career = pd.read_csv('Companies_Career_History.csv')
companies_cross = pd.concat([companies_person, companies_career]).drop_duplicates('Leadership People Id')

ong_person = pd.read_csv('Ong_Person.csv')
ong_career = pd.read_csv('Ong_Career_History.csv')
ong = pd.concat([ong_person, ong_career]).drop_duplicates('Leadership People Id')

state_local = pd.read_csv('State_Local_Cross.csv')
state_local_1 = pd.read_csv('State&Local Government_Current Job Title_Last.csv')
state_local = pd.concat([state_local, state_local_1]).drop_duplicates('Leadership People Id')

congress_person = pd.read_csv('Congress_Person.csv', encoding = 'latin-1')
congress_pastjob = pd.read_csv('Congress_Past_Job.csv')
congress = pd.concat([congress_person, congress_pastjob]).drop_duplicates('Leadership People Id')

nw = pd.read_csv('News_and_Media_Cross.csv')
nw_1 = pd.read_csv('News Media_Current Job Title_Last.csv', encoding = 'latin-1')
nw_2 = pd.read_csv('NewsMedia_Past Job Title_part1_Last.csv', encoding = 'latin-1')
nw = pd.concat([nw, nw_1, nw_2]).drop_duplicates('Leadership People Id')

law = pd.read_csv('Law_and_Lobbying_Cross.csv').drop_duplicates('Leadership People Id')
fed = pd.read_csv('Federal_Government_Cross.csv').drop_duplicates('Leadership People Id')

courts = pd.read_csv('Court_Final Cross_Past&Current_Last.csv', encoding = 'latin-1').drop_duplicates('Leadership People Id')

In [5]:
panel(ong, 'ong_Panel.csv')
panel(companies_cross, 'companies_Panel.csv')
panel(state_local, 'State_Local_Panel.csv')
panel(congress, 'Congress_Panel.csv')
panel(nw, 'News_and_Media_Panel.csv')
panel(law, 'Law_and_Lobbying_Panel.csv')
panel(fed,'Federal_Government_Panel.csv')
panel(courts, 'Courts_Panel.csv')

100%|██████████████████████████████████████████████████████████████████████████████| 830/830 [00:00<00:00, 4910.12it/s]


In [15]:
companies = pd.read_csv('companies_Panel.csv')
ong = pd.read_csv('ong_Panel.csv')
congress = pd.read_csv('Congress_Panel.csv')
fg = pd.read_csv('Federal_Government_Panel.csv')
ll = pd.read_csv('Law_and_Lobbying_Panel.csv')
nw = pd.read_csv('News_and_Media_Panel.csv')
state_local = pd.read_csv('State_Local_Panel.csv')
courts = pd.read_csv('Courts_Panel.csv')

In [16]:
rf = pd.read_csv('FG2_1.csv')
cols = list(rf.columns) + ['Current Start Date', 'Other Current', 'End Date']

final = pd.concat([congress, fg, ll, nw, state_local, companies, ong, courts])
final = final[cols]
print(f'The shape of the final dataset is: {final.shape}')
final.head()

The shape of the final dataset is: (3509991, 43)


,Leadership People Id,Prefix,First Name,Middle Name,Nickname,Last Name,Maiden Name,Suffix,Credentials,Title,...,Country,Education,Careers,Other Current Roles,Political Affiliation,Gender,Start Date,Current Start Date,Other Current,End Date
0,2167960.0,NaN,Heather,NaN,NaN,Millett,NaN,NaN,NaN,Legislative Assistant,...,USA,Campbell University 2020 BA; American Universi...,"Legislative Assistant, Office of Senator Chuck...",NaN,NaN,NaN,2023,01/2023,0.0,NaN
1,2167960.0,NaN,Heather,NaN,NaN,Millett,NaN,NaN,NaN,President,...,USA,Campbell University 2020 BA; American Universi...,"Legislative Assistant, Office of Senator Chuck...",NaN,NaN,NaN,2019,01/2023,0.0,2020.0
2,2167960.0,NaN,Heather,NaN,NaN,Millett,NaN,NaN,NaN,Management Temp,...,USA,Campbell University 2020 BA; American Universi...,"Legislative Assistant, Office of Senator Chuck...",NaN,NaN,NaN,2020,01/2023,0.0,2020.0
3,2167960.0,NaN,Heather,NaN,NaN,Millett,NaN,NaN,NaN,Accounts Payable Specialist,...,USA,Campbell University 2020 BA; American Universi...,"Legislative Assistant, Office of Senator Chuck...",NaN,NaN,NaN,2020,01/2023,0.0,2021.0
4,2167960.0,NaN,Heather,NaN,NaN,Millett,NaN,NaN,NaN,Accounting Technician III,...,USA,Campbell University 2020 BA; American Universi...,"Legislative Assistant, Office of Senator Chuck...",NaN,NaN,NaN,2021,01/2023,0.0,2021.0


In [19]:
#final.to_csv('Final_Panel.csv')

# Final Cross Dataset

In [20]:
companies_person = pd.read_csv('Companies_Person.csv')
companies_career = pd.read_csv('Companies_Career_History.csv')
companies_cross = pd.concat([companies_person, companies_career])

ong_person = pd.read_csv('Ong_Person.csv')
ong_career = pd.read_csv('Ong_Career_History.csv')
ong = pd.concat([ong_person, ong_career])

state_local = pd.read_csv('State_Local_Cross.csv')
state_local_1 = pd.read_csv('State&Local Government_Current Job Title_Last.csv')

congress_person = pd.read_csv('Congress_Person.csv', encoding = 'latin-1')
congress_pastjob = pd.read_csv('Congress_Past_Job.csv')
congress_final = pd.read_csv('Congress_Final_Cross (Current&Past)_Last.csv', encoding = 'latin-1')
congress = pd.concat([congress_person, congress_pastjob, congress_final])

nw = pd.read_csv('News_and_Media_Cross.csv')
nw_1 = pd.read_csv('News Media_Current Job Title_Last.csv', encoding = 'latin-1')
nw_2 = pd.read_csv('NewsMedia_Past Job Title_part1_Last.csv', encoding = 'latin-1')

law = pd.read_csv('Law_and_Lobbying_Cross.csv')
fed = pd.read_csv('Federal_Government_Cross.csv')

courts = pd.read_csv('Court_Final Cross_Past&Current_Last.csv', encoding = 'latin-1')

rf = pd.read_csv('FG2_1.csv')

final_cross = pd.concat([companies_cross, ong, state_local, state_local_1, congress, law, fed, nw, nw_1, nw_2, courts])
final_cross.drop_duplicates('Leadership People Id', inplace = True)
final_cross = final_cross[list(rf.columns)]

print(final_cross.shape)
final_cross.head()

(937552, 40)


,Leadership People Id,Prefix,First Name,Middle Name,Nickname,Last Name,Maiden Name,Suffix,Credentials,Title,...,State,Zip,Region,Country,Education,Careers,Other Current Roles,Political Affiliation,Gender,Start Date
0,2163501.0,NaN,Pamela,NaN,NaN,Abdy,NaN,NaN,NaN,Co-Chairperson and Chief Executive Officer,...,CA,91522,NaN,USA,NaN,"Intern, Jersey Films; Receptionist and Assista...","Governor, Board of Governors, Academy of Motio...",NaN,F,07/2022
1,2180210.0,NaN,Charles,E.,NaN,Abernethy,NaN,NaN,NaN,"Director, Space and Hypersonic Systems",...,VA,22209,NaN,USA,NaN,NaN,NaN,NaN,M,NaN
2,1991889.0,NaN,Timothy,NaN,Tim,Alden,NaN,NaN,NaN,"Managing Director, Aerospace, Defense and Gove...",...,NY,10019,NaN,USA,NaN,"Managing Director, Aerospace, Defense and Fede...",NaN,NaN,M,04/2019
3,830670.0,NaN,Kelly,M.,NaN,Alexander,NaN,Jr.,NaN,Member,...,NC,27601-1096,NaN,USA,University of North Carolina at Chapel Hill 19...,"Member, North Carolina House Homeland Security...","Representative, Office of Representative Kelly...",Democrat,M,01/2019
4,2206923.0,NaN,Joseph,NaN,Joe,Alfieri,NaN,NaN,NaN,"Vice President and President, Space & Defense ...",...,NY,14059,NaN,USA,Rochester Institute of Technology 2004 BS; Roc...,NaN,NaN,NaN,M,03/01/2023


In [9]:
#final_cross.to_csv('Final_Cross.csv')

In [4]:
#final_cross = pd.read_csv('Final_Cross.csv').drop('Unnamed: 0', axis = 1)
panel(final_cross, 'Final_Panel.csv')

100%|██████████████████████████████████████████████████████████████████████████| 56545/56545 [00:22<00:00, 2494.07it/s]


In [3]:
final_cross = pd.read_csv('Final_Cross.csv').drop('Unnamed: 0', axis = 1)
print(final_cross.shape)

(937552, 40)


In [4]:
#panel(final_cross, 'Final_Panel_2.csv')

100%|██████████████████████████████████████████████████████████████████████████| 56545/56545 [00:22<00:00, 2551.23it/s]


# Panel Checks

## Check 1

In [22]:
final_panel = pd.read_csv('Final_Panel.csv')
final_panel.shape

(2819376, 44)

In [23]:
#Final is the file generated by concatenating the single panellized communities
final.shape[0] - final_panel.shape[0]

690615

In [30]:
final[final.duplicated()] #In final there are duplicates

,Leadership People Id,Prefix,First Name,Middle Name,Nickname,Last Name,Maiden Name,Suffix,Credentials,Title,...,Country,Education,Careers,Other Current Roles,Political Affiliation,Gender,Start Date,Current Start Date,Other Current,End Date
48,415996.0,NaN,Alma,Shealey,NaN,Adams,NaN,NaN,NaN,Member,...,USA,North Carolina Agricultural and Technical Stat...,"School Board Member, Guilford County Schools, ...","Member, House Hunger Caucus; Incumbent, Alma A...",Democrat,F,NaN,01/2023,0.0,NaN
52,415996.0,NaN,Alma,Shealey,NaN,Adams,NaN,NaN,NaN,Member,...,USA,North Carolina Agricultural and Technical Stat...,"School Board Member, Guilford County Schools, ...","Member, House Hunger Caucus; Incumbent, Alma A...",Democrat,F,NaN,01/2023,0.0,NaN
57,415996.0,NaN,Alma,Shealey,NaN,Adams,NaN,NaN,NaN,Member,...,USA,North Carolina Agricultural and Technical Stat...,"School Board Member, Guilford County Schools, ...","Member, House Hunger Caucus; Incumbent, Alma A...",Democrat,F,NaN,01/2023,0.0,NaN
59,415996.0,NaN,Alma,Shealey,NaN,Adams,NaN,NaN,NaN,Member,...,USA,North Carolina Agricultural and Technical Stat...,"School Board Member, Guilford County Schools, ...","Member, House Hunger Caucus; Incumbent, Alma A...",Democrat,F,NaN,01/2023,0.0,NaN
62,415996.0,NaN,Alma,Shealey,NaN,Adams,NaN,NaN,NaN,Member,...,USA,North Carolina Agricultural and Technical Stat...,"School Board Member, Guilford County Schools, ...","Member, House Hunger Caucus; Incumbent, Alma A...",Democrat,F,2013,01/2023,0.0,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107156,1699412.0,NaN,Kevin,NaN,NaN,Lewis,NaN,NaN,NaN,Law Clerk,...,USA,NaN,NaN,NaN,NaN,M,2022,NaN,NaN,NaN
107634,1774397.0,NaN,Morris,I.,NaN,Onyewuchi,NaN,NaN,NaN,Immigration Judge,...,USA,NaN,NaN,NaN,NaN,M,2016,NaN,NaN,NaN
108639,1792326.0,NaN,Elena,NaN,NaN,Aguilar,NaN,NaN,NaN,Judicial Assistant,...,USA,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN
108969,2157318.0,NaN,JD,NaN,NaN,Peacock,NaN,II,NaN,County Clerk and Comptroller,...,USA,NaN,NaN,NaN,NaN,M,2014,NaN,NaN,NaN


In [31]:
final_panel[final_panel.duplicated()] #In final_panel there are no duplicates, therefore it's the correct one

,Unnamed: 0,Leadership People Id,Prefix,First Name,Middle Name,Nickname,Last Name,Maiden Name,Suffix,Credentials,...,Country,Education,Careers,Other Current Roles,Political Affiliation,Gender,Start Date,Current Start Date,Other Current,End Date


## Check 2

In [5]:
cross = pd.read_csv('Final_Cross.csv').drop('Unnamed: 0', axis = 1)
print(cross.shape)

(2819376, 43)
(937552, 40)


In [74]:
cross.loc[59, 'Careers'].split(';')

['Communications Specialist, IBM',
 ' Director, Media Relations, L3Harris Technologies, Inc.',
 ' Senior Vice President, Edelman',
 ' Vice President, Communications,']

In [42]:
cross.loc[26, 'Other Current Roles']

'Trustee, Board of Trustees, ULI-the Urban Land Institute; Member, Neeley School of Business International Board of Visitors, M. J. Neeley School of Business, Texas Christian University; President, Hillwood Properties, Hillwood Development Company, LLC'

In [6]:
panel_2 = pd.read_csv('Final_Panel.csv').drop('Unnamed: 0', axis = 1)
print(panel_2.shape)

(3520773, 44)


In [7]:
panel_2.loc[panel_2['Leadership People Id'] == 752334.0, ['Title', 'Organization Name (Parent)', 'Full Job']]

,Title,Organization Name (Parent),Full Job
179,Alliance Air Services President,"City of Fort Worth, Texas","Alliance Air Services President, City of Fort ..."
180,Trustee,ULI-the Urban Land Institute,"Trustee, Board of Trustees, ULI-the Urban Land..."
181,Member,Texas Christian University,"Member, Neeley School of Business Internation..."
182,President,Hillwood Development Company LLC,"President, Hillwood Properties, Hillwood Deve..."


In [8]:
panel_2.loc[panel_2['Leadership People Id'] == 715900.0, ['Title', 'Organization Name (Parent)', 'Full Job']]

,Title,Organization Name (Parent),Full Job
531,"Director, Global Public Relations","L3Harris Technologies, Inc.","Director, Global Public Relations, L3Harris Te..."
532,Communications Specialist,IBM,"Communications Specialist, IBM"
533,Director,L3Harris Technologies Inc.,"Director, Media Relations, L3Harris Technolog..."
534,Senior Vice President,Edelman,"Senior Vice President, Edelman"
535,Vice President,NaN,"Vice President, Communications,"


## Check 3 - Problematic Organizations

In [2]:
final_panel = pd.read_csv('Final_Panel_2.csv').drop('Unnamed: 0', axis = 1)
print(final_panel.shape)
final_panel.head()

(3501449, 44)


,Leadership People Id,Prefix,First Name,Middle Name,Nickname,Last Name,Maiden Name,Suffix,Credentials,Title,...,Education,Careers,Other Current Roles,Political Affiliation,Gender,Start Date,Full Job,Current Start Date,Other Current,End Date
0,2163501.0,NaN,Pamela,NaN,NaN,Abdy,NaN,NaN,NaN,Co-Chairperson and Chief Executive Officer,...,NaN,"Intern, Jersey Films; Receptionist and Assista...","Governor, Board of Governors, Academy of Motio...",NaN,F,07/2022,"Co-Chairperson and Chief Executive Officer, Wa...",07/2022,0.0,NaN
1,2163501.0,NaN,Pamela,NaN,NaN,Abdy,NaN,NaN,NaN,Intern,...,NaN,"Intern, Jersey Films; Receptionist and Assista...","Governor, Board of Governors, Academy of Motio...",NaN,F,NaN,"Intern, Jersey Films",07/2022,0.0,NaN
2,2163501.0,NaN,Pamela,NaN,NaN,Abdy,NaN,NaN,NaN,Receptionist and Assistant to Danny DeVito,...,NaN,"Intern, Jersey Films; Receptionist and Assista...","Governor, Board of Governors, Academy of Motio...",NaN,F,NaN,"Receptionist and Assistant to Danny DeVito, J...",07/2022,0.0,NaN
3,2163501.0,NaN,Pamela,NaN,NaN,Abdy,NaN,NaN,NaN,President,...,NaN,"Intern, Jersey Films; Receptionist and Assista...","Governor, Board of Governors, Academy of Motio...",NaN,F,NaN,"President, Jersey Films",07/2022,0.0,NaN
4,2163501.0,NaN,Pamela,NaN,NaN,Abdy,NaN,NaN,NaN,Executive Vice President,...,NaN,"Intern, Jersey Films; Receptionist and Assista...","Governor, Board of Governors, Academy of Motio...",NaN,F,NaN,"Executive Vice President, Paramount Pictures ...",07/2022,0.0,NaN


In [3]:
problem_org = []
problem_ids = []
empty_ids = []

for i in tqdm(range(final_panel.shape[0])):
    if str(final_panel.loc[i, 'Organization Name (Parent)']) != 'nan':
        if len(final_panel.loc[i, 'Organization Name (Parent)']) <= 5:
            problem_org.append(final_panel.loc[i, 'Organization Name (Parent)'])
            problem_ids.append(final_panel.loc[i, 'Leadership People Id'])
            if final_panel.loc[i, 'Organization Name (Parent)'] == '  ':
                empty_ids.append(final_panel.loc[i, 'Leadership People Id'])

100%|█████████████████████████████████████████████████████████████████████| 3501449/3501449 [03:13<00:00, 18110.19it/s]


In [5]:
counter = Counter(problem_org)
problem_orgs = pd.DataFrame()
problem_orgs['Org'] = counter.keys()
problem_orgs['Num'] = counter.values()

length = []
for k in counter.keys():
    length.append(len(k))

problem_orgs['Length'] = length

print(problem_orgs['Num'].sum())
problem_orgs = problem_orgs.sort_values('Num', ascending = False).reset_index(drop = True)
problem_orgs.loc[0:30, :]

27743


,Org,Num,Length
0,Ohio,1898,5
1,CNN,1736,5
2,IBM,1510,5
3,EY,1171,4
4,NPR,1074,5
5,CNN,889,4
6,P.A.,816,5
7,CNN,720,3
8,N.A.,683,5
9,EY,607,3


# Communities Representatives

## Titles Representatives

In [9]:
def top_30(data):
    freq_final = pd.DataFrame()
    for i in range(len(data.columns)):
        col = data.columns[i]
        if col != 'Leadership People Id':
            freq = pd.DataFrame()
            dm = data.groupby(col).count().reset_index()[[col, 'Leadership People Id']].sort_values('Leadership People Id', ascending = False)
            if len(dm[col]) > 30:
                freq[col] = dm.loc[:30, col]
                freq[f'{col}_freq'] = dm.loc[:30, 'Leadership People Id']
                freq.reset_index(drop = True, inplace = True)
                freq.loc[30, col] = 'Other'
                freq.loc[30, f'{col}_freq'] = dm.loc[:30, 'Leadership People Id'].sum()
                freq[f'{col}_perc'] = freq[f'{col}_freq']*100/freq[f'{col}_freq'].sum()
                freq.reset_index(drop = True, inplace = True)
                freq_final = pd.concat([freq_final, freq], axis = 1)
            else:
                freq[col] = dm[col]
                freq[f'{col}_freq'] = dm['Leadership People Id']
                freq[f'{col}_perc'] = freq[f'{col}_freq']*100/freq[f'{col}_freq'].sum()
                freq.reset_index(drop = True, inplace = True)
                freq_final = pd.concat([freq_final, freq], axis = 1)
            

    return freq_final.iloc[:31, :]

In [2]:
ll = pd.read_csv('Law_and_Lobbying_Cross.csv').drop_duplicates('Leadership People Id')
fg = pd.read_csv('Federal_Government_Cross.csv').drop_duplicates('Leadership People Id')

In [53]:
ll_1 = ll.groupby('Title').count()['Leadership People Id'].reset_index().rename({'Leadership People Id':'Count'}, axis = 1).sort_values('Count', ascending = False)
ll_1['Perc'] = ll_1['Count']*100/len(ll['Leadership People Id'])
ll_1.head()

,Title,Count,Perc
2130,Partner,4401,15.350541
530,Co-Chair,2684,9.361702
139,Chair,2137,7.453784
2033,Member,1333,4.649459
1962,Managing Partner,1089,3.798396


In [59]:
fg_1 = fg.groupby('Title').count()['Leadership People Id'].reset_index().rename({'Leadership People Id':'Count'}, axis = 1).sort_values('Count', ascending = False)
fg_1['Perc'] = fg_1['Count']*100/len(fg['Leadership People Id'])
fg_1.head()

,Title,Count,Perc
9957,Director,6541,6.903212
17599,Member,5946,6.275263
8848,Deputy Director,1967,2.075924
4607,Chief,1621,1.710764
5339,Chief of Staff,814,0.859076


## Random Subsample Check

In [3]:
folder = 'C:/Users/jing/Documents/Bocconi/DSBA/Leadership Connect/Final Dataset/Communities_Representatives'
files = os.listdir(folder)
print(files)

fg_trial = pd.read_csv(folder + '/' + files[0]).drop_duplicates('Leadership People Id')
ll1_trial = pd.read_csv(folder + '/' + files[1]).drop_duplicates('Leadership People Id')
ll2_trial = pd.read_csv(folder + '/' + files[2]).drop_duplicates('Leadership People Id')
ll3_trial = pd.read_csv(folder + '/' + files[3]).drop_duplicates('Leadership People Id')

ll_trial = pd.concat([ll1_trial, ll2_trial, ll3_trial])

print(fg_trial.shape)
print(ll_trial.shape)

['fg.csv', 'll_1.csv', 'll_2.csv', 'll_3.csv']
(8335, 40)
(14608, 40)


In [4]:
fg_down = set(fg_trial['Leadership People Id'])
fg_check = set(pd.merge(fg_trial, fg, on = 'Leadership People Id')['Leadership People Id'])

ll_down = set(ll_trial['Leadership People Id'])
ll_check = set(pd.merge(ll_trial, ll, on = 'Leadership People Id')['Leadership People Id'])

In [5]:
print(len(fg_check.intersection(fg_down))*100/len(fg_down))
print(len(ll_check.intersection(ll_down))*100/len(ll_down))

96.85662867426515
98.05805256295889


In [8]:
#Check for patterns in missings

fg_missing = fg_trial[~fg_trial['Leadership People Id'].isin(fg['Leadership People Id'])]
ll_missing = ll_trial[~ll_trial['Leadership People Id'].isin(ll['Leadership People Id'])]

print(fg_missing.shape)
print(ll_missing.shape)

(262, 40)
(283, 40)


In [19]:
fg_miss_perc = top_30(fg_missing)
fg_miss_perc[['Zip', 'Zip_perc']]

,Zip,Zip_perc
0,22314,19.655172
1,20993-0002,4.827586
2,20230,2.758621
3,22203-2114,2.758621
4,20530,2.758621
5,20410,2.068966
6,21244-1850,1.724138
7,20585,1.379310
8,20210,1.379310
9,20520,1.379310


In [20]:
ll_missing_perc = top_30(ll_missing)
ll_missing_perc[['Zip', 'Zip_perc']]

,Zip,Zip_perc
0,20004,2.008929
1,19103-6998,1.562500
2,44113,1.339286
3,20037-1301,1.339286
4,92101,1.116071
5,10001-8602,1.116071
6,75201,0.892857
7,19103-2921,0.892857
8,10017,0.892857
9,90067-3101,0.892857
